# ZXNB

In [3]:
# init
from pyquda.utils import gauge_utils
from pyquda.field import LatticeFermion
from pyquda import core, pyqcu, mpi
import os
import sys
from time import perf_counter
import cupy as cp
test_dir = os.path.dirname(os.path.abspath("./"))
sys.path.insert(0, os.path.join(test_dir, ".."))
os.environ["QUDA_RESOURCE_PATH"] = ".cache"
latt_size = [16, 16, 16, 32]
grid_size = [1, 1, 1, 1]
Lx, Ly, Lz, Lt = latt_size
Nd, Ns, Nc = 4, 4, 3
Gx, Gy, Gz, Gt = grid_size
latt_size = [Lx//Gx, Ly//Gy, Lz//Gz, Lt//Gt]
Lx, Ly, Lz, Lt = latt_size
Vol = Lx * Ly * Lz * Lt
mpi.init(grid_size)
device_latt_tmp0 = LatticeFermion(latt_size, cp.zeros(
    (Lt, Lz, Ly, Lx, Ns, Nc), cp.complex128))
device_latt_tmp1 = LatticeFermion(latt_size, cp.zeros(
    (Lt, Lz, Ly, Lx, Ns, Nc), cp.complex128))
latt_shape = (Lt, Lz, Ly, Lx//2, Ns, Nc)
param = pyqcu.QcuParam()
param.lattice_size = latt_size
mpi_dslash = core.getDslash(latt_size, -3.5, 0, 0, anti_periodic_t=False)
kappa = 0.125
U = gauge_utils.gaussGauge(latt_size, 0)
mpi_dslash.loadGauge(U)

Creating Gaussian distrbuted Lie group field with sigma = 1.000000e-01


In [4]:
# give ans first
ans_e = cp.random.random(latt_shape) + 1j * \
    cp.random.random(latt_shape)  # ans_e
ans_o = cp.random.random(latt_shape) + 1j * \
    cp.random.random(latt_shape)  # ans_o

In [ ]:
# give x_o, b__o, r, r_tilde, p, v, s, t
x_o = cp.random.random(latt_shape) + 1j * cp.random.random(latt_shape)  # x_o
b_e = cp.zeros(latt_shape, cp.complex128)
b_o = cp.zeros(latt_shape, cp.complex128)
b__o = cp.zeros(latt_shape, cp.complex128)
r = cp.zeros(latt_shape, cp.complex128)
r_tilde = cp.zeros(latt_shape, cp.complex128)
p = cp.zeros(latt_shape, cp.complex128)
s = cp.zeros(latt_shape, cp.complex128)
v = cp.zeros(latt_shape, cp.complex128)
t = cp.zeros(latt_shape, cp.complex128)
# give r_norm2, MAX_ITER, TOL, rho_prev, rho, alpha, omega, beta, kappa
r_norm2 = 0
MAX_ITER = 1e3
TOL = 1e-16
rho_prev = 1
rho = 0
alpha = 1
omega = 1
beta = 0
kappa = 0.125

In [6]:
# define dslash
def dslash_eo(src_o):
    device_latt_tmp1.data[1, :] = src_o
    pyqcu.dslashQcu(device_latt_tmp0.even_ptr, device_latt_tmp1.odd_ptr,
                    U.data_ptr, param, 0)  # D_eo
    return device_latt_tmp0.data[0, :]


def dslash_oe(src_e):
    device_latt_tmp1.data[0, :] = src_e
    pyqcu.dslashQcu(device_latt_tmp0.odd_ptr, device_latt_tmp1.even_ptr,
                    U.data_ptr, param, 1)  # D_oe
    return device_latt_tmp0.data[1, :]


def dslash(src_o):
    return src_o-kappa**2*dslash_oe(dslash_eo(src_o))

In [7]:
# give b'_o(b__0)
b__o = (ans_o-kappa*dslash_oe(ans_e))+kappa*dslash_oe((ans_e-kappa*dslash_eo(ans_o)))

wilson dslash total time: (without malloc free memcpy) :0.002198099 sec
wilson dslash total time: (without malloc free memcpy) :0.000018929 sec
wilson dslash total time: (without malloc free memcpy) :0.000022518 sec


In [8]:
def dot(a, b):
    cp.cuda.runtime.deviceSynchronize()
    return cp.inner(a.flatten().conjugate(), b.flatten())


def diff(a, b):
    cp.cuda.runtime.deviceSynchronize()
    return cp.linalg.norm(a - b) / cp.linalg.norm(b)

In [9]:
# bistabcg
r = b__o - dslash(x_o)
r_tilde = r

wilson dslash total time: (without malloc free memcpy) :0.000025879 sec
wilson dslash total time: (without malloc free memcpy) :0.000024511 sec


In [10]:
t0 = perf_counter()
for i in range(1, int(MAX_ITER)):
    # print("## rho:", rho)
    rho = dot(r_tilde, r)
    # print("## beta:", beta)
    beta = (rho / rho_prev) * (alpha / omega)
    p = r + (p - v * omega) * beta
    # v = A * p
    v = dslash(p)
    # print("## alpha:", alpha)
    alpha = rho / dot(r_tilde, v)
    s = r - v * alpha
    # t = A * s
    t = dslash(s)
    # print("## omega:", omega)
    omega = dot(t, s) / dot(t, t)
    x_o = x_o + p * alpha + s * omega
    r = s - t * omega
    r_norm2 = cp.linalg.norm(r)
    print("##{}# r_norm2:{}".format(i, r_norm2))
    # break
    if (r_norm2 < TOL or i == MAX_ITER - 1):
        print("### turns:", i)
        break
    rho_prev = rho
t1 = perf_counter()

wilson dslash total time: (without malloc free memcpy) :0.000025954 sec
wilson dslash total time: (without malloc free memcpy) :0.000029910 sec
wilson dslash total time: (without malloc free memcpy) :0.000037082 sec
wilson dslash total time: (without malloc free memcpy) :0.000046852 sec
##1# r_norm2:52.74404265113345
wilson dslash total time: (without malloc free memcpy) :0.000029111 sec
wilson dslash total time: (without malloc free memcpy) :0.000023864 sec
wilson dslash total time: (without malloc free memcpy) :0.000017244 sec
wilson dslash total time: (without malloc free memcpy) :0.000027035 sec
##2# r_norm2:22.783426919692268
wilson dslash total time: (without malloc free memcpy) :0.000032320 sec
wilson dslash total time: (without malloc free memcpy) :0.000040175 sec
wilson dslash total time: (without malloc free memcpy) :0.000082881 sec
wilson dslash total time: (without malloc free memcpy) :0.000020703 sec
##3# r_norm2:52.35297374751848
wilson dslash total time: (without malloc 

In [ ]:
print("### ans_o = ", ans_o[0, 0, 0, 0, :, :])
print("### x_o = ", x_o[0, 0, 0, 0, :, :])
print('## difference: ', diff(x_o, ans_o))
print(f'Qcu time: {t1 - t0} sec')

### ans_o =  [[0.3424012 +0.3532274j  0.36839124+0.21647012j 0.11741085+0.82981132j]
 [0.82698589+0.99077119j 0.36590288+0.25654971j 0.69557489+0.10385907j]
 [0.7083333 +0.28584504j 0.45771385+0.59800529j 0.82383404+0.98715991j]
 [0.22063985+0.5950945j  0.53853839+0.08843996j 0.95424866+0.9103208j ]]
### x_o =  [[0.3424012 +0.3532274j  0.36839124+0.21647012j 0.11741085+0.82981132j]
 [0.82698589+0.99077119j 0.36590288+0.25654971j 0.69557489+0.10385907j]
 [0.7083333 +0.28584504j 0.45771385+0.59800529j 0.82383404+0.98715991j]
 [0.22063985+0.5950945j  0.53853839+0.08843996j 0.95424866+0.9103208j ]]
## difference:  8.782607159092754e-16
Qcu time: 5.534150187999785 sec


In [ ]:
x_e = ans_e-kappa*dslash_eo(ans_o)+kappa * \
    dslash_eo(x_o)  # equal to x_o == ans_o
print(diff(x_e, ans_e))

4.987696310498938e-16
